# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import os 
import requests
from dotenv import load_dotenv,find_dotenv

_= load_dotenv(find_dotenv())

OLLAMA_MODEL = os.getenv("OLLAMA_MODEL")
OLLAMA_API_URL = os.getenv("OLLAMA_API_URL")

print(OLLAMA_MODEL,OLLAMA_API_URL)

gemma3:1b http://localhost:11434


In [2]:
def get_completion(prompt,model=OLLAMA_MODEL,temperature=0):
    """
    Generates reponses from ollama LLM
    """
    url = f"{OLLAMA_API_URL}/api/generate"
    payload = {
        "model":model,
        "prompt":prompt,
        "temperature":temperature,
        "stream":False,
    }
    response = requests.post(url,json=payload)
    if response.status_code!=200:
        raise Exception(f"Ollama API error : {response.status_code} {response.text}")
    data = response.json()
    return data['response']
    

In [5]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="gemma3:1b", temperature=0.9)
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [6]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [7]:
chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_15268\1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [8]:
product = "Queen Size Sheet Set"
chain.run(product)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_15268\550859008.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'Okay, let\'s brainstorm some names for a company that makes Queen Size sheet sets. Here\'s a breakdown of options, categorized by the vibe they evoke, with explanations of why they might work:\n\n**1. Emphasizing Comfort & Luxury:**\n\n* **Velvet Dreams:** Suggests a luxurious and dreamy experience.\n* **The Soft Spot:**  Plays on the Queen Size size and the feeling of comfort.\n* **Serene Sheets:** Highlights the peaceful and relaxing feeling of the sheets.\n* **Cloud Comfort:**  Evokes a light, airy, and comfortable feeling.\n* **Haven Sheets:** Positions the sheets as a safe and comfortable place to rest.\n\n**2. Highlighting the Size & Design:**\n\n* **Queen Size Luxe:** Straightforward and emphasizes the key product.\n* **The Size Story:**  Suggests a focus on creating beautiful and well-designed sheets.\n* **Pocketed Perfection:** A bit more playful, focusing on the unique aspect of Queen Size.\n* **Round & Plush:**  Captures the key visual element of the sheets.\n\n**3.  Sophis

## SimpleSequentialChain

In [9]:
from langchain.chains import SimpleSequentialChain

In [14]:
llm = OllamaLLM(model="gemma3:1b", temperature=0.9)
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [17]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Okay, let's brainstorm some names for a company that makes Queen Size Sheet Sets. Here's a breakdown of ideas, categorized by the feeling they evoke, along with a little explanation of why they might work:

**1. Classic & Comfort Focused:**

*   **The Cozy Comforts:** Simple, highlights the feeling of the sheets.
*   **Soft Embrace Textiles:** Emphasizes the softness and comfort.
*   **Serene Sleep Solutions:** Appeals to the desire for a restful night's sleep.
*   **The Comfort Weaver:**  A bit more evocative and suggests skilled craftsmanship.

**2. Playful & Evocative:**

*   **Royal Rest:**  Combines the Queen Size name with the benefit of rest.
*   **The Velvet Dream:**  Uses a luxurious material and a dreamy feeling.
*   **Slumberly:** A cute and memorable name.
*   **Twilight Threads:**  Connects to the nighttime and the soft texture.

**3. Modern & Stylish:**

*   **The Linen Standard:**  Focuses on the quality of the fabric (line

"Okay, great! Let’s refine the recommendations based on your feedback.\n\n**Here’s my updated response, incorporating your feedback:**\n\n**Overall, the company is aiming for a luxurious, comfortable, and subtly stylish brand. Think high-quality, beautiful, and a little bit indulgent. The target audience is likely affluent homeowners or individuals who prioritize comfort and relaxation.**\n\n**Based on your Top 3 recommendations, and considering the brand aesthetic, here’s a revised list of recommendations, with a bit more explanation:**\n\n1.  **The Cozy Comforts:** (Still excellent - core benefit, easy to understand) – **Rationale:** It’s instantly relatable and speaks to the primary benefit of the product.\n\n2.  **Royal Rest:** (Excellent - catchy, memorable, and emphasizes the desired outcome) – **Rationale:**  It’s a strong, aspirational name that conveys a sense of luxury and tranquility.  It’s also memorable.\n\n3.  **Velvet Dream:** (Strong - Evokes a luxurious, soft, and plea

## SequentialChain

In [18]:
from langchain.chains import SequentialChain

In [19]:

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [20]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [21]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [22]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [23]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [25]:
review =  """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

"""
overall_chain(review)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_15268\3804859531.py:14: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\n',
 'English_Review': 'Please provide the text you would like me to translate into English. I need the text of the review to perform the requested extraction. 😊\n',
 'summary': 'Please paste the review text you’d like me to translate! 😊\n',
 'followup_message': 'Okay, here’s the review text:\n\n“Honestly, this coffee shop is a total disaster. The latte art was atrocious – like, completely misshapen blobs. And the barista seemed completely disinterested, barely making eye contact. The atmosphere was also incredibly loud – I couldn’t even have a conversation! Seriously, I wouldn’t recomme

## Router Chain

In [26]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [27]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [28]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [29]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [30]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [31]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

<>:1: SyntaxWarning: invalid escape sequence '\ '
<>:1: SyntaxWarning: invalid escape sequence '\ '
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15268\3710404048.py:1: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \


In [32]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [33]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\ASUS\AppData\Local\Temp\ipykernel_15268\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [34]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Alright, let’s tackle black body radiation.\n\nSimply put, **black body radiation is the electromagnetic radiation emitted by an object when it’s heated.**  \n\nHere’s a breakdown of why it’s important:\n\n*   **It's a fundamental process:**  Everything that emits radiation – even perfect, still objects – does so.\n*   **It’s all about wavelength and frequency:**  The *color* of light emitted isn’t determined by the color itself; it's determined by the object's temperature.\n*   **The Stefan-Boltzmann Law:** This law describes how much energy a black body emits per unit area per unit time.  It’s a cornerstone of understanding this phenomenon.\n\n**Essentially, it's the 'heat glow' of an object.**\n\nDo you want me to delve into a specific aspect, like the relationship to temperature, or perhaps how it’s used in astrophysics?"

In [35]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
physics: {'input': 'what is 2 + 2'}
> Finished chain.


'That’s a fundamental question! 2 + 2 equals 4. \n\nIt’s a basic arithmetic operation. 😊 \n\nDo you have any other questions?'

In [36]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
physics: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Okay, that’s a fantastic question! It’s a really important one, and the answer is surprisingly elegant.\n\nEvery cell in your body contains DNA – deoxyribonucleic acid – because it’s the fundamental blueprint for *everything* you are.  Here’s the breakdown:\n\n*   **DNA is the Genetic Code:** DNA’s primary function is to store and transmit genetic information. Think of it like a really long instruction manual for building and operating a living thing.\n\n*   **Inheritance:**  Our genes – the instructions in DNA – are passed down from parents to offspring.  Without DNA, there would be no continuity of traits.\n\n*   **Cellular Differentiation:**  Each cell in your body has a unique set of DNA. This is crucial for different cell types to perform specialized functions – a muscle cell, a nerve cell, a skin cell – all have their own unique DNA.\n\nSo, DNA is absolutely essential for the survival and function of *every* cell. It’s the key to life as we know it!\n\n---\n\nDo you have any oth